# Experimental Analysis - Flow, Thermal, Energy

This notebook demonstrates experimental analysis capabilities for thermomagnetic generator research. Learn how to:

- **Analyze flow characteristics** (connectivity, tortuosity, resistance)
- **Analyze thermal performance** (thermal resistance, heat transfer coefficient)
- **Estimate energy conversion** (power output, efficiency)
- **Perform integrated analysis** combining all three aspects
- **Optimize for performance** using structure-property relationships

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
1. Analyze flow connectivity and tortuosity
2. Compute thermal resistance and heat transfer coefficients
3. Estimate power output and energy conversion efficiency
4. Understand structure-performance relationships
5. Optimize designs for thermomagnetic generators

## ⚠️ Prerequisites

- **Notebook 01**: Basic understanding of loading and segmenting volumes
- **Notebook 03**: Understanding of morphological analysis
- **Required packages**: Same as previous notebooks
- **Segmented volume**: Binary segmented volume ready for analysis

## 📖 Usage

1. Run all cells to initialize the widgets
2. Load a segmented volume
3. Configure flow, thermal, and energy parameters
4. Run analyses individually or together
5. Explore integrated results and performance metrics


## 1. Setup and Imports


In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
from typing import Dict, List, Optional, Tuple, Any

warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check for ipywidgets
try:
    import ipywidgets as widgets
    from ipywidgets import HBox, VBox, Output, Tab, interactive
    from IPython.display import display, clear_output, HTML
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("❌ ipywidgets not available!")
    print("   Install with: pip install ipywidgets")

# Find project root
current_dir = Path().resolve()
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'src').exists():
    project_root = current_dir
else:
    project_root = current_dir

# Add to path
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

print("📦 Experimental Analysis - Flow, Thermal, Energy")
print(f"   Project root: {project_root}")
print(f"   Widgets available: {WIDGETS_AVAILABLE}")


📦 Experimental Analysis - Flow, Thermal, Energy
   Project root: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/XCT_Thermomagnetic_Analysis
   Widgets available: True


## 2. Load Framework Modules


In [ ]:
# Load experimental analysis modules
try:
    from src.analyzer import XCTAnalyzer
    from src.experimental.flow_analysis import (
        analyze_flow_connectivity, compute_tortuosity,
        estimate_flow_resistance, comprehensive_flow_analysis
    )
    from src.experimental.thermal_analysis import (
        compute_thermal_resistance, estimate_heat_transfer_coefficient,
        estimate_temperature_gradient
    )
    from src.experimental.energy_conversion import (
        estimate_power_output, calculate_energy_conversion_efficiency,
        comprehensive_energy_conversion_analysis
    )
    from src.core.metrics import compute_all_metrics
    from src.utils.utils import load_volume, normalize_path
    
    print("✅ All modules loaded successfully")
except ImportError as e:
    print(f"❌ Error loading modules: {e}")
    import traceback
    traceback.print_exc()
    raise


✅ All modules loaded successfully


## 3. Interactive Experimental Analysis Dashboard

Use the interactive widgets below to perform flow, thermal, and energy conversion analysis.


In [3]:
if not WIDGETS_AVAILABLE:
    print("❌ Cannot create widgets - ipywidgets not available")
else:
    print("🎨 Creating interactive widgets...")
    
    # Initialize state
    analyzer = None
    current_volume = None
    analysis_results = {}
    
    # ============================================
    # Section 1: Data Loading
    # ============================================
    
    file_path_text = widgets.Text(
        value='',
        placeholder='Enter file path to segmented volume',
        description='File Path:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    file_format_dropdown = widgets.Dropdown(
        options=['Auto-detect', 'DICOM', 'TIFF', 'RAW', 'NIfTI', 'NumPy'],
        value='Auto-detect',
        description='Format:',
        style={'description_width': 'initial'}
    )
    
    voxel_size_x = widgets.FloatText(value=0.1, description='Voxel X (mm):', style={'description_width': 'initial'})
    voxel_size_y = widgets.FloatText(value=0.1, description='Voxel Y (mm):', style={'description_width': 'initial'})
    voxel_size_z = widgets.FloatText(value=0.1, description='Voxel Z (mm):', style={'description_width': 'initial'})
    
    load_button = widgets.Button(
        description='📂 Load Volume',
        button_style='primary',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    volume_info_display = widgets.HTML(
        value="<p><i>No volume loaded</i></p>",
        layout=widgets.Layout(height='100px', overflow='auto')
    )
    
    # ============================================
    # Section 2: Flow Analysis Parameters
    # ============================================
    
    flow_direction = widgets.Dropdown(
        options=['X', 'Y', 'Z'],
        value='Z',
        description='Flow Direction:',
        style={'description_width': 'initial'}
    )
    
    flow_velocity = widgets.FloatText(
        value=0.1,
        description='Flow Velocity (m/s):',
        style={'description_width': 'initial'}
    )
    
    flow_rate = widgets.FloatText(
        value=0.001,
        description='Flow Rate (m³/s):',
        style={'description_width': 'initial'}
    )
    
    channel_mean_diameter = widgets.FloatText(
        value=1.0,
        description='Mean Channel Diameter (mm):',
        style={'description_width': 'initial'}
    )
    
    analyze_flow_button = widgets.Button(
        description='🌊 Analyze Flow',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )
    
    flow_results_display = widgets.HTML(
        value="<p><i>No flow analysis</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    flow_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Section 3: Thermal Analysis Parameters
    # ============================================
    
    thermal_conductivity = widgets.FloatText(
        value=50.0,
        description='Thermal Conductivity (W/m·K):',
        style={'description_width': 'initial'}
    )
    
    material_density = widgets.FloatText(
        value=7850.0,
        description='Density (kg/m³):',
        style={'description_width': 'initial'}
    )
    
    specific_heat = widgets.FloatText(
        value=500.0,
        description='Specific Heat (J/kg·K):',
        style={'description_width': 'initial'}
    )
    
    analyze_thermal_button = widgets.Button(
        description='🔥 Analyze Thermal',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )
    
    thermal_results_display = widgets.HTML(
        value="<p><i>No thermal analysis</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    thermal_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Section 4: Energy Conversion Parameters
    # ============================================
    
    temperature_gradient = widgets.FloatText(
        value=50.0,
        description='Temperature Gradient (K):',
        style={'description_width': 'initial'}
    )
    
    curie_temperature = widgets.FloatText(
        value=100.0,
        description='Curie Temperature (°C):',
        style={'description_width': 'initial'}
    )
    
    magnetic_susceptibility = widgets.FloatText(
        value=1000.0,
        description='Magnetic Susceptibility:',
        style={'description_width': 'initial'}
    )
    
    saturation_magnetization = widgets.FloatText(
        value=1.0,
        description='Saturation Magnetization (T):',
        style={'description_width': 'initial'}
    )
    
    analyze_energy_button = widgets.Button(
        description='⚡ Analyze Energy',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )
    
    energy_results_display = widgets.HTML(
        value="<p><i>No energy analysis</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    energy_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Section 5: Integrated Analysis
    # ============================================
    
    analyze_all_button = widgets.Button(
        description='🎯 Analyze All',
        button_style='success',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    integrated_results_display = widgets.HTML(
        value="<p><i>No integrated analysis</i></p>",
        layout=widgets.Layout(height='250px', overflow='auto')
    )
    
    integrated_visualization = Output(layout=widgets.Layout(height='500px'))
    
    # ============================================
    # Progress and Status
    # ============================================
    
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description='Progress:',
        style={'bar_color': '#2ecc71'},
        layout=widgets.Layout(width='400px')
    )
    
    status_display = widgets.HTML(
        value="<p>Ready</p>",
        layout=widgets.Layout(height='60px', overflow='auto')
    )
    
    print("✅ Widgets created successfully!")


🎨 Creating interactive widgets...
✅ Widgets created successfully!


## 4. Widget Callbacks and Functions


In [4]:
if WIDGETS_AVAILABLE:
    
    def load_volume_callback(button):
        """Load segmented volume"""
        global analyzer, current_volume
        
        file_path = file_path_text.value.strip()
        if not file_path:
            status_display.value = "<p style='color: red;'>Please enter a file path</p>"
            return
        
        file_path_obj = Path(file_path)
        if not file_path_obj.exists():
            data_path = project_root / 'data' / file_path
            if data_path.exists():
                file_path_obj = data_path
            else:
                status_display.value = f"<p style='color: red;'>File not found: {file_path}</p>"
                return
        
        status_display.value = "<p>Loading volume...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            analyzer = XCTAnalyzer(voxel_size=voxel_size, target_unit='mm')
            progress_bar.value = 40
            
            analyzer.load_volume(str(file_path_obj), normalize=True)
            current_volume = analyzer.volume
            progress_bar.value = 80
            
            info_html = f"""
            <h4>Volume Information</h4>
            <p><b>Shape:</b> {analyzer.volume.shape}</p>
            <p><b>Voxel Size:</b> {voxel_size} mm</p>
            <p><b>Volume Size:</b> {analyzer.volume.nbytes / (1024**2):.2f} MB</p>
            """
            volume_info_display.value = info_html
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Volume loaded successfully!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error loading volume: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def analyze_flow_callback(button):
        """Analyze flow characteristics"""
        global analyzer, current_volume, analysis_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Analyzing flow...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            direction = flow_direction.value.lower()
            
            # Connectivity analysis
            connectivity = analyze_flow_connectivity(current_volume, flow_direction=direction, voxel_size=voxel_size)
            progress_bar.value = 40
            
            # Tortuosity
            tortuosity_result = compute_tortuosity(current_volume, flow_direction=direction, voxel_size=voxel_size)
            progress_bar.value = 60
            
            # Flow resistance
            channel_geometry = {'mean_diameter': float(channel_mean_diameter.value) / 1000.0}  # Convert to m
            flow_conditions = {
                'velocity': float(flow_velocity.value),
                'flow_rate': float(flow_rate.value)
            }
            resistance = estimate_flow_resistance(
                current_volume, channel_geometry, flow_conditions, voxel_size=voxel_size
            )
            progress_bar.value = 80
            
            analysis_results['flow'] = {
                'connectivity': connectivity,
                'tortuosity': tortuosity_result,
                'resistance': resistance
            }
            
            # Display results
            html = f"""
            <h4>🌊 Flow Analysis Results</h4>
            <p><b>Flow Direction:</b> {flow_direction.value}</p>
            <p><b>Connected:</b> {'✅ Yes' if connectivity['connected'] else '❌ No'}</p>
            <p><b>Tortuosity:</b> {tortuosity_result.get('tortuosity', 'N/A'):.3f if tortuosity_result.get('tortuosity') else 'N/A'}</p>
            <p><b>Path Length:</b> {connectivity.get('path_length_physical', 'N/A'):.3f if connectivity.get('path_length_physical') else 'N/A'} mm</p>
            <p><b>Number of Components:</b> {connectivity.get('n_components', 0)}</p>
            <p><b>Pressure Drop:</b> {resistance.get('pressure_drop_kpa', 'N/A'):.3f if resistance.get('pressure_drop_kpa') else 'N/A'} kPa</p>
            <p><b>Reynolds Number:</b> {resistance.get('reynolds_number', 'N/A'):.1f if resistance.get('reynolds_number') else 'N/A'}</p>
            """
            flow_results_display.value = html
            
            # Visualize
            with flow_visualization:
                clear_output()
                fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                
                # Tortuosity visualization
                if tortuosity_result.get('tortuosity'):
                    axes[0].bar(['Tortuosity'], [tortuosity_result['tortuosity']], color='steelblue', alpha=0.7)
                    axes[0].axhline(1.0, color='red', linestyle='--', label='Ideal (1.0)')
                    axes[0].set_ylabel('Tortuosity', fontsize=11)
                    axes[0].set_title('Flow Path Tortuosity', fontsize=12, fontweight='bold')
                    axes[0].legend()
                    axes[0].grid(True, alpha=0.3, axis='y')
                
                # Pressure drop vs flow rate
                if resistance.get('pressure_drop_kpa'):
                    # Create flow rate range for visualization
                    flow_rates = np.linspace(0.0001, 0.01, 50)
                    # Simplified: pressure drop proportional to flow rate squared
                    pressure_drops = resistance['pressure_drop_kpa'] * (flow_rates / flow_rate.value) ** 2
                    axes[1].plot(flow_rates * 1000, pressure_drops, 'b-', linewidth=2)
                    axes[1].scatter([flow_rate.value * 1000], [resistance['pressure_drop_kpa']], 
                                  s=200, color='red', zorder=5, label='Current')
                    axes[1].set_xlabel('Flow Rate (L/s)', fontsize=11)
                    axes[1].set_ylabel('Pressure Drop (kPa)', fontsize=11)
                    axes[1].set_title('Pressure Drop vs Flow Rate', fontsize=12, fontweight='bold')
                    axes[1].legend()
                    axes[1].grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Flow analysis complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error analyzing flow: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def analyze_thermal_callback(button):
        """Analyze thermal characteristics"""
        global analyzer, current_volume, analysis_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Analyzing thermal...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            
            material_properties = {
                'thermal_conductivity': float(thermal_conductivity.value),
                'density': float(material_density.value),
                'specific_heat': float(specific_heat.value)
            }
            
            flow_conditions = {
                'velocity': float(flow_velocity.value),
                'flow_rate': float(flow_rate.value)
            }
            
            # Thermal resistance
            thermal_resistance = compute_thermal_resistance(
                current_volume, voxel_size, material_properties, flow_conditions
            )
            progress_bar.value = 50
            
            # Heat transfer coefficient
            htc_result = estimate_heat_transfer_coefficient(
                current_volume, voxel_size, flow_conditions, material_properties
            )
            progress_bar.value = 80
            
            analysis_results['thermal'] = {
                'resistance': thermal_resistance,
                'htc': htc_result
            }
            
            # Display results
            html = f"""
            <h4>🔥 Thermal Analysis Results</h4>
            <p><b>Thermal Conductivity:</b> {thermal_conductivity.value} W/m·K</p>
            <p><b>Conduction Resistance:</b> {thermal_resistance.get('conduction_resistance', 'N/A'):.6f if thermal_resistance.get('conduction_resistance') else 'N/A'} K/W</p>
            <p><b>Convection Resistance:</b> {thermal_resistance.get('convection_resistance', 'N/A'):.6f if thermal_resistance.get('convection_resistance') else 'N/A'} K/W</p>
            <p><b>Total Resistance:</b> {thermal_resistance.get('total_resistance', 'N/A'):.6f if thermal_resistance.get('total_resistance') else 'N/A'} K/W</p>
            <p><b>Thermal Conductance:</b> {thermal_resistance.get('thermal_conductance', 0):.6f} W/K</p>
            <p><b>Heat Transfer Coefficient:</b> {htc_result.get('htc', 0):.2f} W/m²·K</p>
            """
            thermal_results_display.value = html
            
            # Visualize
            with thermal_visualization:
                clear_output()
                fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                
                # Resistance breakdown
                resistances = []
                labels = []
                if thermal_resistance.get('conduction_resistance'):
                    resistances.append(thermal_resistance['conduction_resistance'])
                    labels.append('Conduction')
                if thermal_resistance.get('convection_resistance'):
                    resistances.append(thermal_resistance['convection_resistance'])
                    labels.append('Convection')
                
                if resistances:
                    axes[0].bar(labels, resistances, color=['steelblue', 'coral'], alpha=0.7)
                    axes[0].set_ylabel('Resistance (K/W)', fontsize=11)
                    axes[0].set_title('Thermal Resistance Breakdown', fontsize=12, fontweight='bold')
                    axes[0].grid(True, alpha=0.3, axis='y')
                
                # HTC vs flow velocity
                velocities = np.linspace(0.01, 1.0, 50)
                # Simplified: HTC proportional to velocity^0.8 (typical for forced convection)
                htc_values = htc_result.get('htc', 10) * (velocities / flow_velocity.value) ** 0.8
                axes[1].plot(velocities, htc_values, 'b-', linewidth=2)
                axes[1].scatter([flow_velocity.value], [htc_result.get('htc', 10)], 
                              s=200, color='red', zorder=5, label='Current')
                axes[1].set_xlabel('Flow Velocity (m/s)', fontsize=11)
                axes[1].set_ylabel('Heat Transfer Coefficient (W/m²·K)', fontsize=11)
                axes[1].set_title('HTC vs Flow Velocity', fontsize=12, fontweight='bold')
                axes[1].legend()
                axes[1].grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Thermal analysis complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error analyzing thermal: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def analyze_energy_callback(button):
        """Analyze energy conversion"""
        global analyzer, current_volume, analysis_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Analyzing energy conversion...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            
            material_properties = {
                'thermal_conductivity': float(thermal_conductivity.value),
                'density': float(material_density.value),
                'specific_heat': float(specific_heat.value),
                'curie_temperature': float(curie_temperature.value),
                'magnetic_susceptibility': float(magnetic_susceptibility.value),
                'saturation_magnetization': float(saturation_magnetization.value)
            }
            
            flow_conditions = {
                'velocity': float(flow_velocity.value),
                'flow_rate': float(flow_rate.value)
            }
            
            # Power output
            power_result = estimate_power_output(
                current_volume,
                voxel_size,
                temperature_gradient=float(temperature_gradient.value),
                material_properties=material_properties,
                flow_conditions=flow_conditions
            )
            progress_bar.value = 60
            
            # Energy conversion efficiency
            heat_input = power_result.get('heat_transfer_rate', 100.0)  # W
            efficiency_result = calculate_energy_conversion_efficiency(
                current_volume, voxel_size, heat_input
            )
            progress_bar.value = 80
            
            analysis_results['energy'] = {
                'power': power_result,
                'efficiency': efficiency_result
            }
            
            # Display results
            html = f"""
            <h4>⚡ Energy Conversion Analysis Results</h4>
            <p><b>Temperature Gradient:</b> {temperature_gradient.value} K</p>
            <p><b>Power Output:</b> {power_result.get('power_output', 0):.6f} W ({power_result.get('power_output_mw', 0):.3f} mW)</p>
            <p><b>Power Density:</b> {power_result.get('power_density_kw_per_m3', 0):.6f} kW/m³</p>
            <p><b>Heat Transfer Rate:</b> {power_result.get('heat_transfer_rate', 0):.3f} W</p>
            <p><b>Energy Conversion Efficiency:</b> {efficiency_result.get('efficiency', 0):.2%}</p>
            <p><b>Magnetic Factor:</b> {power_result.get('magnetic_factor', 0):.3f}</p>
            <p><b>Heat Transfer Factor:</b> {power_result.get('heat_transfer_factor', 0):.3f}</p>
            """
            energy_results_display.value = html
            
            # Visualize
            with energy_visualization:
                clear_output()
                fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                
                # Power output vs temperature gradient
                temp_gradients = np.linspace(10, 100, 50)
                # Simplified: power proportional to temperature gradient
                power_values = power_result.get('power_output', 0) * (temp_gradients / temperature_gradient.value)
                axes[0].plot(temp_gradients, power_values * 1000, 'b-', linewidth=2)  # Convert to mW
                axes[0].scatter([temperature_gradient.value], [power_result.get('power_output_mw', 0)], 
                              s=200, color='red', zorder=5, label='Current')
                axes[0].set_xlabel('Temperature Gradient (K)', fontsize=11)
                axes[0].set_ylabel('Power Output (mW)', fontsize=11)
                axes[0].set_title('Power Output vs Temperature Gradient', fontsize=12, fontweight='bold')
                axes[0].legend()
                axes[0].grid(True, alpha=0.3)
                
                # Factor contributions
                factors = ['Magnetic', 'Heat Transfer', 'Temperature']
                factor_values = [
                    power_result.get('magnetic_factor', 0),
                    power_result.get('heat_transfer_factor', 0),
                    power_result.get('temperature_factor', 0)
                ]
                axes[1].bar(factors, factor_values, color=['steelblue', 'coral', 'lightgreen'], alpha=0.7)
                axes[1].set_ylabel('Factor Value', fontsize=11)
                axes[1].set_title('Performance Factor Contributions', fontsize=12, fontweight='bold')
                axes[1].set_ylim([0, 1.1])
                axes[1].grid(True, alpha=0.3, axis='y')
                
                plt.tight_layout()
                plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Energy analysis complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error analyzing energy: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def analyze_all_callback(button):
        """Run all analyses together"""
        global analyzer, current_volume, analysis_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Running integrated analysis...</p>"
        progress_bar.value = 10
        
        try:
            # Run all analyses
            analyze_flow_callback(None)
            progress_bar.value = 30
            analyze_thermal_callback(None)
            progress_bar.value = 60
            analyze_energy_callback(None)
            progress_bar.value = 90
            
            # Integrated summary
            if 'flow' in analysis_results and 'thermal' in analysis_results and 'energy' in analysis_results:
                flow = analysis_results['flow']
                thermal = analysis_results['thermal']
                energy = analysis_results['energy']
                
                html = f"""
                <h4>🎯 Integrated Analysis Summary</h4>
                <table style='width:100%; border-collapse: collapse;'>
                <tr style='background-color: #f0f0f0;'>
                    <th style='padding: 8px; text-align: left; border: 1px solid #ddd;'>Metric</th>
                    <th style='padding: 8px; text-align: right; border: 1px solid #ddd;'>Value</th>
                </tr>
                <tr><td style='padding: 8px; border: 1px solid #ddd;'>Flow Connected</td>
                    <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{'✅ Yes' if flow['connectivity']['connected'] else '❌ No'}</td></tr>
                <tr><td style='padding: 8px; border: 1px solid #ddd;'>Tortuosity</td>
                    <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{flow['tortuosity'].get('tortuosity', 'N/A'):.3f if flow['tortuosity'].get('tortuosity') else 'N/A'}</td></tr>
                <tr><td style='padding: 8px; border: 1px solid #ddd;'>Pressure Drop</td>
                    <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{flow['resistance'].get('pressure_drop_kpa', 'N/A'):.3f if flow['resistance'].get('pressure_drop_kpa') else 'N/A'} kPa</td></tr>
                <tr><td style='padding: 8px; border: 1px solid #ddd;'>Thermal Resistance</td>
                    <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{thermal['resistance'].get('total_resistance', 'N/A'):.6f if thermal['resistance'].get('total_resistance') else 'N/A'} K/W</td></tr>
                <tr><td style='padding: 8px; border: 1px solid #ddd;'>Heat Transfer Coefficient</td>
                    <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{thermal['htc'].get('htc', 0):.2f} W/m²·K</td></tr>
                <tr><td style='padding: 8px; border: 1px solid #ddd;'>Power Output</td>
                    <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{energy['power'].get('power_output', 0):.6f} W</td></tr>
                <tr><td style='padding: 8px; border: 1px solid #ddd;'>Power Density</td>
                    <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{energy['power'].get('power_density_kw_per_m3', 0):.6f} kW/m³</td></tr>
                <tr><td style='padding: 8px; border: 1px solid #ddd;'>Efficiency</td>
                    <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{energy['efficiency'].get('efficiency', 0):.2%}</td></tr>
                </table>
                """
                integrated_results_display.value = html
                
                # Visualize integrated results
                with integrated_visualization:
                    clear_output()
                    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                    
                    # Performance metrics comparison
                    metrics = ['Tortuosity', 'Thermal\nResistance\n(×10⁻⁶)', 'Power\nOutput\n(mW)', 'Efficiency\n(%)']
                    values = [
                        flow['tortuosity'].get('tortuosity', 1.0) if flow['tortuosity'].get('tortuosity') else 1.0,
                        thermal['resistance'].get('total_resistance', 0) * 1e6 if thermal['resistance'].get('total_resistance') else 0,
                        energy['power'].get('power_output_mw', 0),
                        energy['efficiency'].get('efficiency', 0) * 100
                    ]
                    axes[0, 0].bar(metrics, values, color=['steelblue', 'coral', 'lightgreen', 'gold'], alpha=0.7)
                    axes[0, 0].set_ylabel('Value', fontsize=11)
                    axes[0, 0].set_title('Performance Metrics Summary', fontsize=12, fontweight='bold')
                    axes[0, 0].grid(True, alpha=0.3, axis='y')
                    
                    # Flow vs Thermal
                    axes[0, 1].scatter([flow['tortuosity'].get('tortuosity', 1.0) if flow['tortuosity'].get('tortuosity') else 1.0],
                                      [thermal['resistance'].get('total_resistance', 0) * 1e6 if thermal['resistance'].get('total_resistance') else 0],
                                      s=200, color='steelblue', alpha=0.7)
                    axes[0, 1].set_xlabel('Tortuosity', fontsize=11)
                    axes[0, 1].set_ylabel('Thermal Resistance (×10⁻⁶ K/W)', fontsize=11)
                    axes[0, 1].set_title('Flow vs Thermal Performance', fontsize=12, fontweight='bold')
                    axes[0, 1].grid(True, alpha=0.3)
                    
                    # Thermal vs Energy
                    axes[1, 0].scatter([thermal['htc'].get('htc', 0)],
                                     [energy['power'].get('power_output_mw', 0)],
                                     s=200, color='coral', alpha=0.7)
                    axes[1, 0].set_xlabel('Heat Transfer Coefficient (W/m²·K)', fontsize=11)
                    axes[1, 0].set_ylabel('Power Output (mW)', fontsize=11)
                    axes[1, 0].set_title('Thermal vs Energy Performance', fontsize=12, fontweight='bold')
                    axes[1, 0].grid(True, alpha=0.3)
                    
                    # Performance factors
                    factors = ['Magnetic', 'Heat Transfer', 'Temperature']
                    factor_values = [
                        energy['power'].get('magnetic_factor', 0),
                        energy['power'].get('heat_transfer_factor', 0),
                        energy['power'].get('temperature_factor', 0)
                    ]
                    axes[1, 1].bar(factors, factor_values, color=['steelblue', 'coral', 'lightgreen'], alpha=0.7)
                    axes[1, 1].set_ylabel('Factor Value', fontsize=11)
                    axes[1, 1].set_title('Energy Conversion Factors', fontsize=12, fontweight='bold')
                    axes[1, 1].set_ylim([0, 1.1])
                    axes[1, 1].grid(True, alpha=0.3, axis='y')
                    
                    plt.tight_layout()
                    plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Integrated analysis complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error in integrated analysis: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    # Attach callbacks
    load_button.on_click(load_volume_callback)
    analyze_flow_button.on_click(analyze_flow_callback)
    analyze_thermal_button.on_click(analyze_thermal_callback)
    analyze_energy_button.on_click(analyze_energy_callback)
    analyze_all_button.on_click(analyze_all_callback)
    
    print("✅ Callback functions attached!")


✅ Callback functions attached!


## 5. Display Interactive Dashboard


In [5]:
if WIDGETS_AVAILABLE:
    
    # Create loading panel
    loading_panel = widgets.VBox([
        widgets.HTML("<h2>📂 Load Segmented Volume</h2>"),
        HBox([
            file_path_text,
            file_format_dropdown
        ]),
        HBox([
            widgets.HTML("<b>Voxel Size:</b>"),
            voxel_size_x,
            voxel_size_y,
            voxel_size_z
        ]),
        HBox([load_button, volume_info_display])
    ])
    
    # Create flow analysis panel
    flow_panel = widgets.VBox([
        widgets.HTML("<h3>🌊 Flow Analysis</h3>"),
        flow_direction,
        HBox([flow_velocity, flow_rate]),
        channel_mean_diameter,
        analyze_flow_button,
        flow_results_display,
        flow_visualization
    ])
    
    # Create thermal analysis panel
    thermal_panel = widgets.VBox([
        widgets.HTML("<h3>🔥 Thermal Analysis</h3>"),
        thermal_conductivity,
        HBox([material_density, specific_heat]),
        analyze_thermal_button,
        thermal_results_display,
        thermal_visualization
    ])
    
    # Create energy analysis panel
    energy_panel = widgets.VBox([
        widgets.HTML("<h3>⚡ Energy Conversion Analysis</h3>"),
        temperature_gradient,
        HBox([curie_temperature, magnetic_susceptibility]),
        saturation_magnetization,
        analyze_energy_button,
        energy_results_display,
        energy_visualization
    ])
    
    # Create integrated analysis panel
    integrated_panel = widgets.VBox([
        widgets.HTML("<h3>🎯 Integrated Analysis</h3>"),
        widgets.HTML("<p>Run all analyses together to see comprehensive performance metrics</p>"),
        analyze_all_button,
        integrated_results_display,
        integrated_visualization
    ])
    
    # Create tabs for organized display
    analysis_tabs = Tab(children=[
        flow_panel,
        thermal_panel,
        energy_panel,
        integrated_panel
    ])
    analysis_tabs.set_title(0, '🌊 Flow')
    analysis_tabs.set_title(1, '🔥 Thermal')
    analysis_tabs.set_title(2, '⚡ Energy')
    analysis_tabs.set_title(3, '🎯 Integrated')
    
    # Create main dashboard
    dashboard = widgets.VBox([
        widgets.HTML("<h1>🔬 Experimental Analysis - Flow, Thermal, Energy</h1>"),
        loading_panel,
        widgets.HTML("<hr>"),
        widgets.HTML("<h2>📊 Analysis</h2>"),
        analysis_tabs,
        widgets.HTML("<hr>"),
        progress_bar,
        status_display
    ])
    
    # Display the dashboard
    display(dashboard)
    print("\n✅ Dashboard displayed! Start analyzing flow, thermal, and energy conversion.")
    print("\n💡 Tips:")
    print("   1. Load a segmented volume")
    print("   2. Configure flow parameters (direction, velocity, channel size)")
    print("   3. Set material properties for thermal analysis")
    print("   4. Configure magnetic properties for energy conversion")
    print("   5. Run individual analyses or use 'Analyze All' for integrated results")
    
else:
    print("❌ Cannot display dashboard - ipywidgets not available")



✅ Dashboard displayed! Start analyzing flow, thermal, and energy conversion.

💡 Tips:
   1. Load a segmented volume
   2. Configure flow parameters (direction, velocity, channel size)
   3. Set material properties for thermal analysis
   4. Configure magnetic properties for energy conversion
   5. Run individual analyses or use 'Analyze All' for integrated results


## 6. Summary

### What We Learned

1. **Flow Analysis**:
   - Flow path connectivity (inlet to outlet)
   - Tortuosity calculation (path complexity)
   - Flow resistance and pressure drop estimation
   - Reynolds number and flow regime analysis

2. **Thermal Analysis**:
   - Thermal resistance (conduction and convection)
   - Heat transfer coefficient estimation
   - Temperature gradient analysis
   - Material property effects

3. **Energy Conversion**:
   - Power output estimation
   - Energy conversion efficiency
   - Power density calculation
   - Magnetic and thermal factor contributions

4. **Integrated Analysis**:
   - Combined flow-thermal-energy performance
   - Structure-performance relationships
   - Optimization insights

### Key Insights

- **Flow connectivity** is essential for functional heat exchangers
- **Low tortuosity** improves flow efficiency but may reduce heat transfer
- **Thermal resistance** depends on both material properties and structure
- **Energy conversion efficiency** is typically 1-5% for thermomagnetic generators
- **Power output** scales with temperature gradient and structure quality

### Next Steps

- **Notebook 05**: Advanced Analysis - Sensitivity and Virtual Experiments
  - Parameter sensitivity analysis
  - Virtual experiment design
  - Process optimization

- **Notebook 06**: Comparative Analysis and Batch Processing
  - Compare multiple samples
  - Batch processing workflows
  - Statistical analysis

### Resources

- [Framework Documentation](../docs/README.md)
- [Experimental Modules](../docs/modules.md#experimental-modules)
- [Flow Analysis](../docs/modules.md#experimentalflow_analysis)
- [Thermal Analysis](../docs/modules.md#experimentalthermal_analysis)
- [Energy Conversion](../docs/modules.md#experimentalenergy_conversion)
